In [8]:
import pandas as pd
import numpy as np
import re

In [10]:
data=pd.read_csv('test.csv',encoding='ISO-8859-1')

In [11]:
df=pd.DataFrame(data)
df.head(6)

,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265.0,470.0,164.0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0
5,726e501993,that`s great!! weee!! visitors!,positive,night,70-100,Antigua and Barbuda,97929.0,440.0,223.0


In [19]:
df.size

48150

In [17]:
df['Cleaned Text'].isnull().sum()

1281

In [22]:
df=df.dropna()

In [23]:
df.isnull().sum()

textID              0
text                0
sentiment           0
Time of Tweet       0
Age of User         0
Country             0
Population -2020    0
Land Area (Km²)     0
Density (P/Km²)     0
Cleaned Text        0
dtype: int64

In [25]:
df.size

35340

In [15]:
# Define the preprocessing function
def preprocess_text(text):
    if isinstance(text, str):  # Check if the value is a string
        # Convert to lowercase
        text = text.lower()

        # Remove URLs
        text = re.sub(r'http\S+', '', text)

        # Remove non-alphanumeric characters except spaces
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

        # Remove extra spaces
        text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply preprocessing to the 'Text' column
df['Cleaned Text'] = df['text'].apply(preprocess_text)

In [16]:
df.head(5)

,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²),Cleaned Text
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0,last session of the day
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0,shanghai is also really exciting precisely sky...
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044.0,2381740.0,18.0,recession hit veronique branquinho she has to ...
3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265.0,470.0,164.0,happy bday
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0,i like it


In [27]:
df1=df.drop(columns=['textID','Time of Tweet','Age of User','Country','Population -2020','Land Area (Km²)','Density (P/Km²)'])

In [55]:
df1.head(10)

,text,sentiment,Cleaned Text
0,Last session of the day http://twitpic.com/67ezh,neutral,last session of the day
1,Shanghai is also really exciting (precisely -...,positive,shanghai is also really exciting precisely sky...
2,"Recession hit Veronique Branquinho, she has to...",negative,recession hit veronique branquinho she has to ...
3,happy bday!,positive,happy bday
4,http://twitpic.com/4w75p - I like it!!,positive,i like it
5,that`s great!! weee!! visitors!,positive,thats great weee visitors
6,I THINK EVERYONE HATES ME ON HERE lol,negative,i think everyone hates me on here lol
7,"soooooo wish i could, but im in school and my...",negative,soooooo wish i could but im in school and mysp...
8,and within a short time of the last clue all ...,neutral,and within a short time of the last clue all o...
9,What did you get? My day is alright.. haven`...,neutral,what did you get my day is alright havent done...


In [76]:
X = df1['Cleaned Text']
y = df1['sentiment']

In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=3000) 
X_tfidf = tfidf_vectorizer.fit_transform(X)


In [78]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [79]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_encoded, test_size=0.2, random_state=42)

In [80]:
from sklearn.svm import SVC  # Support Vector Classifier

model = SVC(kernel='linear')
model.fit(X_train, y_train)


SVC(kernel='linear')

In [81]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
print("Classification Report:\n", report)

confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", confusion)


Accuracy: 0.637906647807638
Classification Report:
               precision    recall  f1-score   support

    negative       0.69      0.53      0.60       207
     neutral       0.57      0.76      0.65       286
    positive       0.75      0.59      0.66       214

    accuracy                           0.64       707
   macro avg       0.67      0.62      0.63       707
weighted avg       0.66      0.64      0.64       707

Confusion Matrix:
 [[109  89   9]
 [ 37 216  33]
 [ 12  76 126]]


In [82]:
import pickle

with open('sentiment_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

In [83]:
with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(tfidf_vectorizer, vectorizer_file)

with open('label_encoder.pkl', 'wb') as label_encoder_file:
    pickle.dump(label_encoder, label_encoder_file)

In [84]:
# loading the pickle files here to make prediction
with open('sentiment_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

with open('tfidf_vectorizer.pkl', 'rb') as vectorizer_file:
    loaded_tfidf_vectorizer = pickle.load(vectorizer_file)

with open('label_encoder.pkl', 'rb') as label_encoder_file:
    loaded_label_encoder = pickle.load(label_encoder_file)


In [96]:
def predict_sentiment(text):
    cleaned_text = preprocess_text(text)  # Apply your preprocessing steps here
    features = loaded_tfidf_vectorizer.transform([cleaned_text])
    sentiment_label_encoded = loaded_model.predict(features)
    sentiment_label = loaded_label_encoder.inverse_transform(sentiment_label_encoded)[0]
    return sentiment_label

test_text = '''
I am vikas
'''
predicted_sentiment = predict_sentiment(test_text)
print(predicted_sentiment)

neutral
